# Install Libraries

In [34]:
# !pip3 install scikit-optimize pycaret[tuners] RiskLabAI==0.0.87 joblib_progress memory_profiler pycaret arch 
# !pip3 install torch
# !pip3 install yfinance

# Import Libraries

In [51]:
from RiskLabAI.features.feature_importance.swefi import *
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Dataset

In [38]:
data = pd.read_csv('real_data.csv', index_col=0)
data = data.dropna()
data = data.sort_values('Date')
data = data.set_index('Date')
data.index = pd.to_datetime(data.index)
data = data.loc[data.index >= '2018-01-01']

In [39]:
y = 'MXWO_Last Price'
y = pd.Series(np.where(data[y].diff() > 0, 1, 0), name='Direction', index=data.index)
X = data.drop(columns=['MXWO_Last Price', 'MXWO_Volume', 'MXWO_SMAVG (15)'])

# SWEFI Parameters

In [41]:
select_n_model = 5

bootstrap_method = SWEFI.stationary_bootstrap

hpo_n_fold=4
hpo_n_iter=25
hpo_metric = 'AUC'
hpo_search_library = 'scikit-optimize'
hpo_search_algorithm = 'bayesian'

n_iteration = 5

percentage = 0.6

# Initialize SWEFI

In [42]:
swefi = SWEFI(X, y, n_fold=10)

,Description,Value
0,Session id,123
1,Target,Direction
2,Target type,Binary
3,Original data shape,"(1565, 34)"
4,Transformed data shape,"(1565, 34)"
5,Transformed train set shape,"(1549, 34)"
6,Transformed test set shape,"(16, 34)"
7,Numeric features,33
8,Preprocess,True
9,Imputation type,simple


# Select Models

In [43]:
swefi.select_models(select_n_model=select_n_model)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,MLP Classifier,0.6314,0.6672,0.6880,0.6531,0.6694,0.2533,0.2546,1.4760
9,SVM - Radial Kernel,0.6275,0.6681,0.8351,0.6165,0.7092,0.2223,0.2432,0.1290
3,Linear Discriminant Analysis,0.6249,0.6595,0.7818,0.6243,0.6941,0.2248,0.2346,0.0120
5,Ridge Classifier,0.6236,0.6606,0.7853,0.6222,0.6942,0.2215,0.2321,0.0100
4,Logistic Regression,0.6229,0.6605,0.7723,0.6241,0.6902,0.2221,0.2308,0.0120
2,Gradient Boosting Classifier,0.6191,0.6397,0.7308,0.6282,0.6750,0.2204,0.2255,0.3240
6,Random Forest Classifier,0.6152,0.6342,0.7094,0.6306,0.6674,0.2149,0.2173,0.1800
8,Extra Trees Classifier,0.6146,0.6326,0.7059,0.6307,0.6657,0.2140,0.2166,0.0900
7,Quadratic Discriminant Analysis,0.5894,0.5994,0.7639,0.5956,0.6692,0.1491,0.1579,0.0150
1,Decision Tree Classifier,0.5546,0.5507,0.5957,0.5893,0.5915,0.1017,0.1021,0.2880


# Fine-Tune Selected Models

In [44]:
swefi.fine_tune_selected_models(
    hpo_n_fold=hpo_n_fold,
    hpo_n_iter=hpo_n_iter,
    hpo_metric=hpo_metric,
    hpo_search_algorithm=hpo_search_algorithm,
    hpo_search_library=hpo_search_library
);

--------------------------------------------------------------------------------
MLPClassifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6521,0.6917,0.7630,0.6545,0.7046,0.2874,0.2924
1,0.6408,0.6862,0.7476,0.6461,0.6932,0.2656,0.2698
2,0.6382,0.6780,0.7299,0.6498,0.6875,0.2615,0.2640
3,0.6176,0.6723,0.6445,0.6507,0.6476,0.2296,0.2296
Mean,0.6372,0.6820,0.7213,0.6503,0.6832,0.2610,0.2639
Std,0.0125,0.0075,0.0458,0.0030,0.0215,0.0207,0.0225


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fits
Fitting 4 folds for each of 1 candidates, totalling 4 fi

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6314,0.6935,0.8673,0.6141,0.7191,0.2266,0.2568
1,0.6460,0.6946,0.8381,0.6308,0.7198,0.2643,0.2845
2,0.6021,0.6590,0.7962,0.6022,0.6857,0.1710,0.1838
3,0.6176,0.6476,0.8152,0.6121,0.6992,0.2025,0.2187
Mean,0.6243,0.6737,0.8292,0.6148,0.7059,0.2161,0.2359
Std,0.0163,0.0208,0.0265,0.0103,0.0143,0.0341,0.0381


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
--------------------------------------------------------------------------------
LinearDiscriminantAnalysis
Custom config ...


...

Fitting 4 folds for each of 1 candidates, totalling 4 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
--------------------------------------------------------------------------------
RidgeClassifier
Custom config ...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6392,0.6817,0.8152,0.6300,0.7107,0.2516,0.2667
1,0.6563,0.6913,0.8000,0.6486,0.7164,0.2922,0.3027
2,0.6098,0.6531,0.7630,0.6145,0.6808,0.1938,0.2014
3,0.6072,0.6391,0.7441,0.6157,0.6738,0.1912,0.1967
Mean,0.6281,0.6663,0.7806,0.6272,0.6954,0.2322,0.2419
Std,0.0205,0.0211,0.0283,0.0138,0.0184,0.0422,0.0447


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
--------------------------------------------------------------------------------
LogisticRegression
Custom config ...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6495,0.6816,0.8104,0.6404,0.7155,0.2751,0.2882
1,0.6615,0.6854,0.7905,0.6561,0.7171,0.3048,0.3130
2,0.6098,0.6578,0.7393,0.6190,0.6739,0.1977,0.2026
3,0.6176,0.6450,0.7299,0.6286,0.6754,0.2163,0.2199
Mean,0.6346,0.6674,0.7675,0.6360,0.6955,0.2485,0.2559
Std,0.0215,0.0167,0.0338,0.0139,0.0208,0.0433,0.0460


Fitting 4 folds for each of 1 candidates, totalling 4 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


# Select Univariate-Analysis Methods

In [45]:
swefi.select_univariate_analysis_measurements(measurements=[
        # UAMeasure.SPEARMAN.value,
        # UAMeasure.PEARSON.value,
        # UAMeasure.KENDAL_TAU.value,
        UAMeasure.MUTUAL_INFORMATION.value,
        UAMeasure.ANOVA_F.value,
    ]
)

# Feature Importance Computation of Models

In [46]:
swefi.compute_feature_importance_data(bootstrap_method=bootstrap_method, n_iteration=n_iteration, n_repeats=10);

  0%|          | 0/5 [00:00<?, ?it/s]

# SWEFI Scores Computation

In [47]:
swefi.compute_swefi_scores(percentage=percentage)
swefi_scores = swefi.get_swefi_scores()

# Show Results

In [48]:
swefi_scores.to_csv('result.csv')

In [49]:
import pandas as pd
swefi_scores = pd.read_csv('result.csv', index_col=0)

In [50]:
index = swefi_scores.index
value = swefi_scores['mean(SWEFI)']
error = swefi_scores['std(SWEFI)']

fig = go.Figure(go.Bar(
    y=value,
    x=index,
    error_y=dict(type='data', array=error),
))

fig.update_layout(
    xaxis=dict(
        title='Feature',
        zeroline=True,
        showline=True,
        zerolinecolor='black',
        zerolinewidth=3,
        linecolor='black',
        linewidth=3,
        mirror=True,
        tickangle=45,
        tickfont=dict(
            family='Arial',
            size=18,
            color='black',
        )
    ),
    yaxis=dict(
        title='Importance',
        showgrid=True,
        zeroline=True,
        showline=True,
        gridcolor='black',
        gridwidth=1,
        zerolinecolor='black',
        zerolinewidth=3,
        linecolor='black',
        linewidth=3,
        mirror=True,

        tickfont=dict(
            family='Arial',
            size=18,
            color='black',
        )
    ),
    margin=dict(l=10, r=10, b=10, t=10),
    paper_bgcolor='white',
    plot_bgcolor='lightgrey',
    width=2000,
    height=1500,
    bargap=0.1,
    bargroupgap=0.1,
)


# fig.write_image('swefi_success_time_series_vertical.png')
fig.show()
